In [5]:
%run "../../shared/utilz_includez.ipynb"

In [10]:
from skimage import io, img_as_float, img_as_ubyte
from skimage import color , measure

from scipy import ndimage as nd

import cv2 
from skimage.segmentation import clear_border

## Non-local-means denoising Approach @microscope if not using cv2.median
from skimage.restoration import denoise_nl_means, estimate_sigma

In [11]:
def plot_image( img_array , plotit=True, title=None, cmapd='gray', logit=True):     
    if logit:
        print( f"\n{ '-'*7 } { title if title else type(img_array) } { '-'*7 }" )
        print( f"image.shape = {img_array.shape}" ) 
        print( f"datatype = {img_array.dtype}")
        print( f"min = {np.min(img_array)} , max = {np.max(img_array)}\n" )
    if plotit:
        if cmapd:
            plt.imshow( img_array , cmap=cmapd)
        else:
            plt.imshow( img_array )
    if title:
        plt.title(title)
        
        
        
def fetch_image(imgpath, forceuint8=True):
    img = io.imread( imgpath )
    if forceuint8:
        img = np.uint8(img)
    gimg = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY ) 
    return img, gimg
    
def grid_plot_images( imgz, titlz=None, plotit=True, logit=True, cmapd='gray', nr=1, nc=2):
    nr = np.int( len(imgz) / nc )
    if len(imgz) % 2 != 0:
        nr = nr+1
    for i in range( len(imgz) ):
        plt.subplot(nr, nc, i+1 )
        plot_image( imgz[i], title=titlz[i] if titlz else None , plotit=plotit, logit=logit, cmapd=cmapd); 
        

In [12]:
blue_channel = 2

def nuclei_channel_only(img):
    return img[:, :, blue_channel]

In [13]:
## fetch list of imagez and return as dict
def fetch_imagez_dict( imgpath_list, lbl_prefix="Image", logit=False):
    dict_cell_imagez = {} 
    for i, ip in enumerate(imgpath_list):
        c, g = fetch_image( ip )
        if logit:
            print(f"IP at {i} is okay")
        dict_cell_imagez[f"{lbl_prefix} # {i}"] = (c, g )
        
    return dict_cell_imagez

In [ ]:
## gaussian kernel creator

In [14]:
## clean, segment using watershed and generate properties
kern = np.ones( (3,3), dtype=np.uint8 )

pixels_to_um = 0.5 

radianz = 57.2958


region_propz = ['Label', "Area", "Perimeter", 
     "equivalent_diameter", "orientation",
    "MajorAxisLength", "MinorAxisLength",
     "MinIntensity", "MaxIntensity", "MeanIntensity",
    ]
propz_addz = ["Area_nm", "Perimeter_nm", 
     "equivalent_diameter_nm", 
    "orientation_degrees",
    "MajorAxisLength_nm", "MinorAxisLength_nm",]


def nonlocalmeans_clean(gimg, hfactz=1.15, patchz=5, patch_distz=3, fastz=False):
    sig = np.mean( estimate_sigma(gimg, multichannel=True) ) 
    oimg = denoise_nl_means(gimg, h=hfactz*sig, fast_mode=fastz, patch_size=patchz, patch_distance=patch_distz, multichannel=True)
    return oimg

def clean_prepare_binary( gimg , blursize=3):
    # a. binary threshold 
    otsu, timg = cv2.threshold(gimg, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    # b. erode dilate @ crispier grains
    timg = cv2.erode( timg, kern, iterations=1)
    timg = cv2.dilate( timg, kern, iterations=1)

    # c. median blur pixeletation << should this go first 
    timg = cv2.medianBlur(timg, blursize)
    
    return timg

def watershed_segment(cimg, gimg, thresh_val=0.2, kern=np.ones((3,3), dtype=np.uint8), iterz=1, dilate_iterz=1 ): 
    # a. morphological extra 
    mimg = cv2.morphologyEx( gimg, cv2.MORPH_OPEN, kern, iterations=iterz)
    
    # b. clear border cells/grains at this point 
    mimg = clear_border( mimg )
    
    # c. Identify sure foreground pixels     
    sure_bg = cv2.dilate( mimg, kern, iterations=dilate_iterz) 
    mimg = cv2.distanceTransform( mimg, cv2.DIST_L2, 3)
    
    # d.  Threshold
    _, sure_fg = cv2.threshold(mimg, thresh_val*mimg.max(),  255, 0)
    sure_fg = np.uint8( sure_fg )
    
    # e. gen markers and distinguish unkownz from background 
    _, markerz = cv2.connectedComponents( sure_fg )
    
    unknownz = cv2.subtract( sure_bg, sure_fg)
    markerz = markerz+10
    markerz[ unknownz == 255 ] = 0 
    
    # f. watershed
    watershed = cv2.watershed( cimg, markerz )

    ready_img = cimg.copy()
    ready_img[ markerz == -1] = [0, 255, 255]
    rimg2 = color.label2rgb( markerz, bg_label=0 )
    
    ## return markerz, rgb labeled images
    return markerz, rimg2

def gen_properties(markerz, gimg): 
    clusterz = measure.regionprops(markerz, intensity_image=gimg)
    g = clusterz[0]
    
    ## Create dframe for it
    dd = []
    for g in clusterz:
        gl = []
        for p in region_propz:
            gl.append( g[p] )

        for ap in propz_addz:
            p = "_".join( ap.split("_")[:-1])

            ## Area_nm
            if( p == 'Area'):
                gl.append( g[p] * pixels_to_um**2 )
            ## Orientation_nm
            elif( p == 'orientation'):
                gl.append( g[p]* radianz ) 
            ## anything that's not Intensity
            else:
                gl.append( g[p]* pixels_to_um ) 


    dd.append( gl )
    
    # df = pd.DataFrame.from_records( dd , columns=propz+addz )
    ## return list oobject 
    return dd



    

In [15]:
## Descriptors and Matching


In [ ]:
## Fourier Transform
def run_discrete_fourier_transform(gimg):    
    ## a. generate dft 
    dft = cv2.dft( np.float32(gimg), flags=cv2.DFT_COMPLEX_OUTPUT )

    ## b. by default data is centered around the origin and the origin is at top left corner. So move the origin to the center of the grid
    # shift the zero frequency to center
    dft = np.fft.fftshift( dft)

    ## c. compute the magnitude spectrum 
    kil_zero = 0
    dft_ms = 20 * np.log( (cv2.magnitude(dft[:,:,0], dft[:,:,1]) )+kil_zero )# +1 to avoid div by zero error
    
    return dft, dft_ms